# Data Cleaning and Validation 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 

In [3]:
df = pd.read_csv('Wellcome_APCspend2013.csv', encoding='ISO-8859-1') 
#https://stackoverflow.com/questions/21504319/python-3-csv-file-giving-unicodedecodeerror-utf-8-codec-cant-decode-byte-err 
df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [4]:
df.rename(columns={'PMID/PMCID':'PMID/PMCID','Publisher':'Publisher','Journal title':'Journal_Title','Article title':'Article_Title','COST (£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace=True) 

In [5]:
len(df)

2127

In [6]:
#see how many unique ids we have without cleaning 
print(len(df['PMID/PMCID'].unique()))
print(len(df['Publisher'].unique()))
print(len(df['Journal_Title'].unique()))
print(len(df['Article_Title'].unique()))

1881
299
985
2126


In [7]:
#Wellcome Trust asked institutions to cite the PMC ID (or the PubMed ID if a PMC ID is not known)
##need to try to consolidate the format into either PMID (8 digits) or PMCID (7 digits)

In [105]:
#remove whitespacing, create one coherent capitalization
df['PMID/PMCID'] = df['PMID/PMCID'].str.upper().str.strip()
print(len(df['PMID/PMCID'].unique()))

1874


In [59]:
#replace duplicated values, one coherent capitalization, remove whitespacing 
df['Publisher'] = df['Publisher'].str.replace('ACS (Amercian Chemical Society) Publications', 'ACS') 
df['Publisher'] = df['Publisher'].str.replace('ACS Publications','ACS')
df['Publisher'] = df['Publisher'].str.replace('American Chemical Society','ACS')
df['Publisher'] = df['Publisher'].str.replace('American Society for Biochemistry and Molecular Biolgy', 'American Society for Biochemistry and Molecular Biology')
df['Publisher'] = df['Publisher'].str.replace('American Soc for Biochemistry and Molecular Biology','American Society for Biochemistry and Molecular Biology')
df['Publisher'] = df['Publisher'].str.upper().str.strip()
print(len(df['Publisher'].unique()))

254


In [153]:
#check the values without cleaning 
df['Journal_Title'].value_counts().head(10)

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
Human Molecular Genetics                           18
PLoS Neglected Tropical Diseases                   18
Nature Communications                              17
Neuroimage                                         15
PLoS Genetics                                      15
Name: Journal_Title, dtype: int64

In [60]:
#clean data 
df['Journal_Title'] = df['Journal_Title'].str.upper()
df['Journal_Title'] = df['Journal_Title'].str.replace('', '') 
df['Journal_Title'] = df['Journal_Title'].str.replace(',','')
df['Journal_Title'] = df['Journal_Title'].str.replace(':','')
df['Journal_Title'] = df['Journal_Title'].str.replace('-','')
df['Journal_Title'] = df['Journal_Title'].str.replace('.','')
df['Journal_Title'] = df['Journal_Title'].str.replace('&','and')
df['Journal_Title'] = df['Journal_Title'].str.replace("\)",'')
df['Journal_Title'] = df['Journal_Title'].str.replace("\(",'')
df['Journal_Title'] = df['Journal_Title'].str.strip()
print(len(df['Journal_Title'].unique()))

891


In [61]:
#recheck values 
df['Journal_Title'].value_counts().head(10)

PLOS ONE                                           190
JOURNAL OF BIOLOGICAL CHEMISTRY                     53
NEUROIMAGE                                          29
NUCLEIC ACIDS RESEARCH                              26
PLOS PATHOGENS                                      24
PLOS GENETICS                                       24
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES     22
PLOS NEGLECTED TROPICAL DISEASES                    20
NATURE COMMUNICATIONS                               19
HUMAN MOLECULAR GENETICS                            19
Name: Journal_Title, dtype: int64

In [65]:
df['Article_Title'] = df['Article_Title'].str.upper().str.strip()
print(len(df['Article_Title'].unique()))

2126


In [76]:
#edit cost column
df['Cost']= df['Cost'].str.replace('£', '')
df['Cost']=df['Cost'].str.replace('$', '')
df['Cost']=df['Cost'].astype(float)

# Five most common journals and total articles each

In [77]:
df['Journal_Title'].value_counts().head(10)

PLOS ONE                                           190
JOURNAL OF BIOLOGICAL CHEMISTRY                     53
NEUROIMAGE                                          29
NUCLEIC ACIDS RESEARCH                              26
PLOS PATHOGENS                                      24
PLOS GENETICS                                       24
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES     22
PLOS NEGLECTED TROPICAL DISEASES                    20
NATURE COMMUNICATIONS                               19
HUMAN MOLECULAR GENETICS                            19
Name: Journal_Title, dtype: int64

# Mean, median, and standard deviation of the open-access cost per article for each journal

In [93]:
df.groupby(['Journal_Title'])['Cost'].agg(['mean','median','std']).dropna()

,mean,median,std
Journal_Title,,,
ACS CHEMICAL BIOLOGY,1418.186000,1294.590,507.309560
ACS NANO,668.140000,668.140,35.708892
ACTA CRYSTALLOGRAPHICA SECTION F STRUCTURAL BIOLOGY AND CRYSTALLIZATION COMMUNICATIONS,796.635000,796.635,15.605847
ACTA NEUROPATHOLOGICA,2161.063333,2250.970,244.811798
ADDICTION,2136.225000,2136.225,306.481292
ADVANCES IN EXPERIMENTAL MEDICINE AND BIOLOGY,1928.457500,1928.460,0.005000
AGE,2170.140000,2170.140,237.785868
AIDS,2059.306667,1968.630,281.067979
AIDS CARE,2294.225000,2316.010,126.415851
